#### Filtrado de datos

In [1]:
import pandas as pd

In [6]:
datos_entrada = "train.csv"
datos_filtrados = "beverages.csv"

def filtrarDatos(entrada=datos_entrada, salida=datos_filtrados):
    """
    param:
    datos_entrada: .csv inicial (dataset principal del que se va a filtrar los datos necesarios).
    datos_filtrados: .csv final (datos útiles para la ejecución del proyecto).
    """

    # Intento de lectura del archivo CSV
    try:
        df = pd.read_csv(entrada)
    except FileNotFoundError:  # Archivo no encontrado
        print("Archivo no encontrado:", entrada)  # Mensaje de error
        return None
    except Exception as e:  # Captura de otro tipo de errores
        print("Error al leer el CSV:", e)  # Mensaje de error
        return None

    # Búsqueda y validación de las columnas a usar
    columnasAFiltrar = {"family", "sales", "date"}

    # Valida que las columnas se encuentren en el archivo
    if not columnasAFiltrar.issubset(df.columns):  
        # issubset() se encarga de buscar si cada columna necesaria se encuentra en el archivo
        print("El archivo no contiene las columnas deseadas: ", columnasAFiltrar)  
        return None

    # Busca en la columna "family" todos los registros que tengan el valor "BEVERAGES"
    beverages = df[df["family"] == "BEVERAGES"]  # Filtra únicamente el grupo BEVERAGES
    if "onpromotion" in beverages.columns: # Busca la columna onpromotion en los datos filtrados beverages
        beverages = beverages.drop(columns = ["onpromotion"]) # Elimina la columna onpromotion con drop()
    
    # Guardar los datos filtrados en datos_filtrados.csv
    try:
        beverages.to_csv(salida, index=False)  # Guarda los datos filtrados en un nuevo CSV
    except Exception as e:  # Captura cualquier tipo de error
        print("Error al guardar el archivo.", e)
        return None

    return beverages  # Regresa el dataframe filtrado


# Ejecutar la función
filtrarDatos()

,id,date,store_nbr,family,sales
3,3,2013-01-01,1,BEVERAGES,0.0
36,36,2013-01-01,10,BEVERAGES,0.0
69,69,2013-01-01,11,BEVERAGES,0.0
102,102,2013-01-01,12,BEVERAGES,0.0
135,135,2013-01-01,13,BEVERAGES,0.0
...,...,...,...,...,...
3000726,3000726,2017-08-15,54,BEVERAGES,4332.0
3000759,3000759,2017-08-15,6,BEVERAGES,2678.0
3000792,3000792,2017-08-15,7,BEVERAGES,3999.0
3000825,3000825,2017-08-15,8,BEVERAGES,3626.0
